In [24]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding,Dense,SimpleRNN,Dropout,LSTM
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [14]:
df = pd.read_csv('YoutubeCommentsDataSet.csv')
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [15]:

df.dropna(inplace=True)
df.isnull().sum()

Comment      0
Sentiment    0
dtype: int64

In [16]:
df.Sentiment.value_counts()

Sentiment
positive    11389
neutral      4525
negative     2335
Name: count, dtype: int64

In [17]:
count_0, count_1, count_2 = df.Sentiment.value_counts()
count_0,count_1,count_2

(11389, 4525, 2335)

In [18]:
df_0 = df.loc[df['Sentiment'] == 'positive']
df_1 = df.loc[df['Sentiment'] == 'neutral']
df_2 = df.loc[df['Sentiment'] == 'negative']
df_0.head()

,Comment,Sentiment
2,i will forever acknowledge this channel with t...,positive
4,apple pay is so convenient secure and easy to ...,positive
6,we only got apple pay in south africa in 20202...,positive
8,in the united states we have an abundance of r...,positive
10,wow you really went to town on the psu test ra...,positive


In [19]:
df_1_over = df_1.sample(count_0, replace=True)
df_2_over = df_2.sample(count_0, replace=True)
df_0.shape,df_1_over.shape,df_2_over.shape

((11389, 2), (11389, 2), (11389, 2))

In [22]:
df_final = pd.concat([df_0,df_1_over,df_2_over], axis=0)
df_final.shape

(34167, 2)

In [23]:
df_final.to_csv('YoutubeComments.csv', index=False)

In [ ]:
x=[]
for s in df_final['Comment']:
  result = ''.join([i for i in s if not i.isdigit()])
  x.append(result)

In [ ]:
import re
x =[re.sub('[^a-zA-Z]',' ',words) for words in x]

In [ ]:
voca_len =10000
x_ohe = [one_hot(words,voca_len) for words in df_final['Comment']]

In [ ]:
sent_len = 100
x_pad = sequence.pad_sequences(x_ohe,maxlen=sent_len)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
y_ohe = ohe.fit_transform(df_final[['Sentiment']])
y_ohe

In [ ]:
labels = ohe.get_feature_names_out(['Sentiment'])
print(labels)

In [ ]:
y = pd.DataFrame(y_ohe.toarray(), columns=labels)
y.head()
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_pad,y_final,test_size=0.2,random_state=10, stratify=y_final)

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(voca_len,300,input_length=sent_len))
model.add(SimpleRNN(128,activation='relu',return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(64,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
earlystopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [ ]:
history=model.fit(x_train,y_train,epochs=100,
          validation_split=0.2,
          callbacks=[earlystopping])

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
model.save('SentimentRNN.h5')

In [ ]:
import pickle

with open('y_ohe.pkl' , 'wb') as f:
  pickle.dump(ohe,f)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_la,y_pred))